In [1]:
from flask import Flask, render_template
from flask_cors import CORS
import json 

In [12]:
app = Flask(__name__)

CORS(app,resources={r"/*": {"origins": ["https://turkishfilms.github.io"]}})

@app.route('/')
def home():
    return render_template('index.html')
@app.route('/learn')
def a():
    return render_template('learn.html')
    pass
@app.route('/b')
def b():
    pass
@app.route('/c')
def c():
    pass


if(__name__ == "__main__"):
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/May/2023 14:58:21] "GET /learn HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 14:58:21] "GET /static/css/styles.css HTTP/1.1" 200 -
